In [5]:
import tensorflow as tf
import numpy as np

In [16]:
data = ['안녕하세요', '반갑습니다', '신촌으로와', '저녁먹을까', '이것만하고',
        '감사합니다', '도와주세요', '어디갔었어', '정신좀차려', '제정신이니',
        '어디간거야', '어디가세요', '점심먹었니', '신촌에있어']

In [17]:
def encode_char(ch):
    cho = list('ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ')                    # 19
    joong = list('ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ')              # 21
    jong = list(' ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅊㅋㅌㅍㅎ')  # 28

    d = ord(ch) - 0xAC00
    c = d % 28
    b = (d - c) // 28 % 21
    a = ((d - c) // 28 - b) // 21

    return [int(i == a) for i in range(len(cho))] + [int(i == b) for i in range(len(joong))] + [int(i == c) for i in range(len(jong))] # 68

In [18]:
def decode_char(vec):
    cho = list('ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ')                    # 19
    joong = list('ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ')              # 21
    jong = list(' ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅊㅋㅌㅍㅎ')  # 28

    a = 0
    for i in range(0, 19):
        if vec[i] > vec[a]:
            a = i
    b = 19
    for i in range(19, 40):
        if vec[i] > vec[b]:
            b = i
    b -= 19
    c = 40
    for i in range(40, 68):
        if vec[i] > vec[c]:
            c = i
    c -= 40
    d = (a * 21 + b) * 28 + c
    ch = chr(d + 0xAC00)

    return ch

In [19]:
def make_batch(data):
    input_batch = []
    target_batch = []

    for d in data:
        input_batch.append([encode_char(ch) for ch in d[:-1]])
        target_batch.append(encode_char(d[-1]))

    return input_batch, target_batch

In [20]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
n_step = 4
n_input = n_class = 68

In [21]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None, n_class])
W_xh = tf.Variable(tf.random_normal([n_input, n_hidden]))
W_hh = tf.Variable(tf.random_normal([n_hidden, n_hidden]))
b_h = tf.Variable(tf.random_normal([n_hidden]))
W_hy = tf.Variable(tf.random_normal([n_hidden, n_class]))
b_y = tf.Variable(tf.random_normal([n_class]))
x_tr = tf.transpose(X, [1, 0, 2])
x_stream = tf.unstack(x_tr)
h = tf.Variable(tf.zeros([len(data), n_hidden]))
for x in x_stream:
    h = tf.nn.tanh(tf.matmul(h, W_hh) + tf.matmul(x, W_xh) + b_h)
    y = tf.matmul(h, W_hy) + b_y
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=Y))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)
input_batch, target_batch = make_batch(data)

In [22]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(1, total_epoch + 1):
        _, loss = sess.run([train, cost], feed_dict={X:input_batch, Y:target_batch})
        if epoch % 10: continue
        print(epoch, loss)
    ret = sess.run(y, feed_dict={X:input_batch})
    print('원본:', [d[:-1] + 'ㅁ' for d in data])
    print('예측:', [data[i][:-1] + decode_char(r) for i, r in enumerate(ret)])
    print('정확도:', sum([int(data[i][-1] == decode_char(r)) for i, r in enumerate(ret)]) / len(data))

10 21.29425
20 12.115883
30 7.2888947
40 6.259108
50 5.3768797
60 5.825768
70 5.230594
80 5.662709
90 5.58784
100 6.1576576
원본: ['안녕하세ㅁ', '반갑습니ㅁ', '신촌으로ㅁ', '저녁먹을ㅁ', '이것만하ㅁ', '감사합니ㅁ', '도와주세ㅁ', '어디갔었ㅁ', '정신좀차ㅁ', '제정신이ㅁ', '어디간거ㅁ', '어디가세ㅁ', '점심먹었ㅁ', '신촌에있ㅁ']
예측: ['안녕하세요', '반갑습니다', '신촌으로와', '저녁먹을까', '이것만하고', '감사합니다', '도와주세요', '어디갔었어', '정신좀차려', '제정신이니', '어디간거야', '어디가세요', '점심먹었니', '신촌에있어']
정확도: 1.0
